In [98]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../')

import os

In [2]:
date = '2025-01-19'
data = pd.read_csv('../data/oresund/'+date+'.csv') #Import data
data = data.drop_duplicates() #Drop duplicates (there are many!)

Making a copy to manipulate

In [3]:
data_filter = data.copy()

Filtering out helicopters which are faster than 60 knots (or else it's a spurious incident)

In [4]:
#I filter for helicopter
data_filter = data_filter[(data_filter['SOG'] < 60) | (data_filter['SOG'].isna())]

Resampling the series so there's a minimum five minute interval between measurements
This period is of coursse arbitrary, but seems to be a good compromise between memory and resolution
A ship moves $5/60 \cdot 20 \approx 1.6 km$ when sailing fast

In [91]:
data_temp = pd.DataFrame(columns=data_filter.columns)

for MMSI in data_filter['MMSI'].unique():
    ship = data_filter[data_filter['MMSI'] == MMSI]
    #Creating a series for time filtering
    series = pd.Series(range(len(ship)), index=pd.to_datetime(ship['# Timestamp'], dayfirst=True))
    #Resampling the series to 5 minutes. This gives the index of the last data point in the 5 minute interval
    #The unique is added because if transmission occurs less frequently than 5 mins there will be duplicates
    resampled = series.resample('5min').first().dropna().astype(int)
    #Filtering the data
    ship = ship.iloc[resampled]
    data_temp = pd.concat([data_temp, ship], ignore_index=True)

data_filter = data_temp.copy()

/var/folders/29/jqsnm5js6_sc7663vjsq3mp40000gp/T/ipykernel_9510/3287667159.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_temp = pd.concat([data_temp, ship], ignore_index=True)


Now we filter out buoys and aids to navigation ('base station' and 'AtoN')

In [ ]:
print(data['Type of mobile'].unique())
data_filter = data_filter[(data_filter['Type of mobile']!='Base Station') & (data_filter['Type of mobile']!='AtoN')]
print(data_filter['Type of mobile'].unique())

['Class A' 'Class B' 'Base Station' 'AtoN']
['Class A' 'Class B']
67423


Appending the dataset with distance measure which is signed depending on whether ship is north or south of Oresund (see Trials/Oresund_distance_calc)

In [ ]:
#Defining parameters for oresund bridge
long_or = (12.682873, 12.88866)
lat_or = (55.628996, 55.564952)

#Using a flat earth approximation to calculate the slope (in lat/lon) of the bridge
slope_or = (lat_or[1]-lat_or[0])/(long_or[1] - long_or[0])

#Defining function for the corridor going perpendicular to the bridge
def corridor_lon(lat):
    west = -slope_or*(lat - lat_or[0]) + long_or[0]
    east = -slope_or*(lat - lat_or[1]) + long_or[1]
    return west, east

#Calculating the corridor on the latitudes that all the ships are in
west, east = corridor_lon(data_filter['Latitude'])
in_corridor = data_filter['Longitude'].between(west, east) #If the ship is in the corridor
west_of_corridor = data_filter['Longitude'] < west #If the ship is west of the corridor
east_of_corridor = data_filter['Longitude'] > east #If the ship is east of the corridor

#Calculating the distance
lat_dist = 6378*2*np.pi/360 #Latituda distance in km using radius of earth
lon_dist = lat_dist * np.cos(lat_or[0]*np.pi/180) #Longitude distance in km using radius of earth and latitude of bridge

#"distance" coordinates of each ship (will convert to real distances when subtracted from the bridge)
x = data['Longitude']*lon_dist
y = data['Latitude']*lat_dist

x_or = np.array(long_or)*lon_dist
y_or = np.array(lat_or)*lat_dist

bridge_vec_deg = np.array([1, slope_or]) #Bridge vector in degrees
bridge_vec_dist = bridge_vec_deg * np.array([lon_dist, lat_dist]) #Bridge vector in km
n = np.array([-bridge_vec_dist[1], bridge_vec_dist[0]]) #Normal vector to the bridge vector in km
n = n/np.linalg.norm(n) #Normalized normal vector
perp_dist = (x-x_or[0])*n[0] + (y-y_or[0])*n[1] #Perpendicular distance using projection onto normal vector
above = (perp_dist > 0)*2 - 1 #If the ship is above or below the bridge
dist_west = above*np.sqrt((x-x_or[0])**2 + (y-y_or[0])**2) #Signed distance to the west bridge point
dist_east = above*np.sqrt((x-x_or[1])**2 + (y-y_or[1])**2) #Signed distance to the east bridge point

#If the ship is in corridor use the perpendicular distance, else use the closest distance to the bridge
data['Distance'] = perp_dist * in_corridor + dist_west * west_of_corridor + dist_east * east_of_corridor

Now we create the real dataset \
Every datapoint is a triple with (distance, speed, time to cross)

Loop over all ships and
1) Decide if ship is crossing
2) Find time of crossing
3) Calculate time until crossing
4) Calculate speed
5) Append all relevant datapoints

In [ ]:
final = pd.DataFrame(columns=['Distance', 'SOA', 'TTC'])

for MMSI in data_filter['MMSI'].unique():
    ship = data_filter[data_filter['MMSI'] == MMSI]
    dist = ship['Distance']

Down here are demonstrations of the workings of the code

In [90]:
ship = data[data['MMSI'] == data['MMSI'].values[49]]
series = pd.Series(range(len(ship)), index=pd.to_datetime(ship['# Timestamp'], dayfirst=True))
resampled = series.resample('5min')
ind_1 = resampled.first().dropna().astype(int)
ind_2 = (resampled.count().cumsum()-1).unique()
print(ind_1.values[:15])
print(ship.iloc[ind_1]['# Timestamp'][:15])
print(ship.iloc[ind_2]['# Timestamp'][:15])
print(ship['# Timestamp'][:15])

[ 0  3  7  9 11 12 13 16 19 21 24 27 28 32 35]
124      19/01/2025 00:00:08
6251     19/01/2025 00:06:09
13371    19/01/2025 00:12:57
15785    19/01/2025 00:15:08
25406    19/01/2025 00:24:08
28485    19/01/2025 00:27:10
31475    19/01/2025 00:30:08
37779    19/01/2025 00:36:08
43849    19/01/2025 00:42:10
50235    19/01/2025 00:48:08
53439    19/01/2025 00:51:09
60392    19/01/2025 00:57:10
63752    19/01/2025 01:00:09
70313    19/01/2025 01:06:09
77721    19/01/2025 01:12:57
Name: # Timestamp, dtype: object
874      19/01/2025 00:00:57
9518     19/01/2025 00:09:09
13404    19/01/2025 00:12:59
19997    19/01/2025 00:18:58
25406    19/01/2025 00:24:08
28485    19/01/2025 00:27:10
34724    19/01/2025 00:33:08
40840    19/01/2025 00:39:09
44724    19/01/2025 00:42:58
51100    19/01/2025 00:48:58
57699    19/01/2025 00:54:57
60392    19/01/2025 00:57:10
67066    19/01/2025 01:03:10
73630    19/01/2025 01:09:09
77721    19/01/2025 01:12:57
Name: # Timestamp, dtype: object
124      19/01/20